In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
# from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
df = pd.read_csv("Data\Student Grades\student-mat.csv",sep =";")

In [3]:
# Creating y
y = df.G3

# Removing unwanted features and creating X
Features = ['school', 'age', 'address', 'Pstatus', 'studytime', 'failures', 'paid', 'internet', 'famrel', 'absences', 'G1', 'G2']
X = df[Features]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=0)

# Categorical and numerical columns
categorical_cols = [cname for cname in X.columns if X[cname].nunique()<5 and X[cname].dtype == 'object']

numeric_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float46']]

In [4]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder())])

preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_cols)])

# The model
model = LinearRegression()

# Bundle preprocessing and modeling code in a pipeline
new_pipe = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# cv_scores = -1 * cross_val_score(new_pipe,X,y,cv = 3, scoring = "neg_mean_absolute_error")

In [5]:
new_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['school', 'address',
                                                   'Pstatus', 'paid',
                                                   'internet'])])),
                ('model', LinearRegression())])

In [ ]:
cv_scores = -1 * cross_val_score(new_pipe,X_train,y_train,cv = 5, scoring = "neg_mean_absolute_error")
cv_scores.mean()

In [6]:
# preds = model.predict(X_test)
print(X_train.head())
print(X_test.head())

    school  age address Pstatus  studytime  failures paid internet  famrel  \
1       GP   17       U       T          2         0   no      yes       5   
293     GP   17       R       T          4         0  yes       no       3   
278     GP   18       U       T          2         1   no      yes       2   
255     GP   17       U       T          1         1   no      yes       4   
204     GP   16       R       T          4         0  yes      yes       5   

     absences  G1  G2  
1           4   5   5  
293         6  18  18  
278        15   9   8  
255         2   7   9  
204         6  10  10  
    school  age address Pstatus  studytime  failures paid internet  famrel  \
329     GP   17       U       T          3         0  yes      yes       4   
318     GP   17       R       T          3         0  yes      yes       4   
317     GP   18       U       T          3         0  yes      yes       4   
65      GP   16       U       T          2         0   no      yes       5 

In [7]:
# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_test = X_test.copy()

# Apply ordinal encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[categorical_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_test[categorical_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(categorical_cols, axis=1)
num_X_valid = X_test.drop(categorical_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

c:\Users\ASUS\.virtualenvs\researchvenv\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
# print(OH_X_train.head())
# print(OH_X_valid.head())
print(categorical_cols)

['school', 'address', 'Pstatus', 'paid', 'internet']
